### Nodes-link Inter-relation coefficient

Does coherence is fully explained by local power? This can be adressed via the nodes-link inter-realtion coefficient (NLI):

$NIL_{ij} = \langle H[Z(POW_i)]H[Z(POW_j)]H[Z(COH_{ij})]\rangle + \langle \tilde{H}[Z(POW_i)]\tilde{H}[Z(POW_j)]\tilde{H}[Z(COH_{ij})]\rangle$

where, 

$H[x] = x$, if $x \geq 0$ and $\tilde{H}[x] = -x$, if $x \leq 0$

In [1]:
# Adding GDa to path
import os
import sys

import matplotlib.pyplot as plt
import numba as nb
import numpy as np
import pandas as pd
import xarray as xr

sys.path.insert(1, "/home/vinicius/storage1/projects/GrayData-Analysis")

from tqdm import tqdm

from GDa.flatmap.flatmap import flatmap
from GDa.net.layerwise import compute_network_partition
from GDa.net.temporal import compute_allegiance_matrix
from GDa.net.util import convert_to_adjacency
from GDa.temporal_network import temporal_network

In [2]:
@nb.vectorize([nb.float64(nb.float64)])
def H(x):
    if x >= 0:
        return x
    else:
        return 0


@nb.vectorize([nb.float64(nb.float64)])
def H_tilde(x):
    if x <= 0:
        return -x
    else:
        return 0

#### Loading power and temporal network

In [3]:
_ROOT = os.path.expanduser("~/storage1/projects/GrayData-Analysis")
_RESULTS = os.path.join("Results", "lucy", "141017", "session01")

power_file = "power_tt_1_br_1_at_cue.nc"
coh_sig_file = "coh_k_0.3_multitaper_at_cue_surr.nc"
wt = None

In [4]:
power = xr.load_dataarray(os.path.join(_ROOT, _RESULTS, power_file))
power = power.transpose("roi", "freqs", "trials", "times")

In [5]:
net = temporal_network(
    coh_file=f"coh_k_0.3_multitaper_at_cue.nc",
    coh_sig_file=coh_sig_file,
    wt=wt,
    date="141017",
    trial_type=[1],
    behavioral_response=[1],
)

In [6]:
# Concatenate trials and times
power = power.stack(samples=("trials", "times"))
coh = net.super_tensor.stack(samples=("trials", "times"))

In [7]:
# z-score
Zpower = (power - power.mean("samples")) / power.std("samples")
Zcoh = (coh - coh.mean("samples")) / coh.std("samples")

In [8]:
roi = Zcoh.roi.data
freqs = Zcoh.freqs.data
n_rois = len(roi)
n_freqs = len(freqs)

#### Compute NLI

In [9]:
sources = net.super_tensor.attrs["sources"].astype(int)
targets = net.super_tensor.attrs["targets"].astype(int)
areas = np.asarray(net.super_tensor.attrs["areas"])

In [10]:
nli = np.zeros((Zcoh.shape[0], Zcoh.shape[1]))
for p, (s, t) in enumerate(zip(sources, targets)):
    nli[p] = np.mean(H(Zpower[s]) * H(Zpower[t]) * H(Zcoh[p]), -1) + np.mean(
        H_tilde(Zpower[s]) * H_tilde(Zpower[t]) * H_tilde(Zcoh[p]), -1
    )
nli = xr.DataArray(
    nli,
    dims=("roi", "freqs"),
    name="nli",
    coords={"roi": net.super_tensor.roi.data, "freqs": net.super_tensor.freqs.data},
)

In [11]:
# Create data frame with the data
mean_power = power.mean("samples")
mean_coh = coh.mean("samples")

out = []
for f in range(n_freqs):
    data = np.array(
        [
            areas[sources],
            areas[targets],
            sources,
            targets,
            [freqs[f]] * n_rois,
            nli.isel(freqs=f),
            mean_coh.isel(freqs=f),
        ]
    )
    out += [
        pd.DataFrame(
            data=data.T,
            columns=["roi_s", "roi_t", "s", "t", "f", "nli", "coh_st"],
        )
    ]

out = pd.concat(out, axis=0)

In [13]:
out.head()

,roi_s,roi_t,s,t,f,nli,coh_st
0,a46D,a8B,0,1,3.0,1.1182930733291794,0.05599589
1,a46D,F6,0,2,3.0,0.5020865050912139,0.015063142
2,a46D,a9/46D,0,3,3.0,1.0568761563774955,0.04518061
3,a46D,a8M,0,4,3.0,1.759252767440444,0.020243911
4,a46D,a8B,0,5,3.0,1.4272198888328538,0.026135035
